# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [22]:
sc

Calculation started (calculation_id=00c955cf-af32-8c3c-e1cc-045d7487a28b) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [23]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=90c955cf-b060-3cf3-2feb-2f484ed3eaef) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [24]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=24c955cf-b16d-f7dc-cec7-86b5042b2985) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [25]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'hpavlovich3'

Calculation started (calculation_id=6cc955cf-b295-3d74-ff95-9fa141550a69) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [26]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    df = trips.filter(trips.trip_distance >=2)
    return df

Calculation started (calculation_id=e6c955cf-b3cc-13c3-5305-45ce119751c8) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.3 [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [27]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    lookup_hold = lookup.withColumnRenamed("LocationID", "DOLocationID")

    mtrips = trips.join(lookup_hold, ["DOLocationID"], "outer") \
            .filter(col("Borough") == "Manhattan")

    mtrips = mtrips.groupBy("DOLocationID") \
        .agg(sum("passenger_count").alias("pcount")) \
        .select("DOLocationID", "pcount") \
        .orderBy(desc("pcount")) \
        .limit(20)
    
    return mtrips

Calculation started (calculation_id=bac955cf-b55c-e01e-0c1b-f423e873ea42) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [28]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    # Calculate total trips and average amount
    trips_hold = trips.agg(
        count("*").alias("total_trips"),
        avg("total_amount").alias("avg_amount")
    ).collect()[0]

    total_trips = trips_hold["total_trips"]
    avg_amount = trips_hold["avg_amount"]

    # Calculate count of trips to top 20 destinations
    count_20 = trips.join(mtrips, ["DOLocationID"], "inner").count()

    # Calculate PULocationID stats and weighted profit
    wp = trips.groupBy("PULocationID") \
        .agg(count("*").alias("PU_count"),
             avg("total_amount").alias("PU_avg_amount"))

    wp = wp.withColumn("weighted_profit", 
                       ((col("PU_count") * lit(count_20) * col("PU_avg_amount")) /
                       (lit(total_trips) * lit(total_trips))) \
                       .cast("decimal(24,10)")) \
    .select("PULocationID", "weighted_profit")
    
    return wp

Calculation started (calculation_id=d2c955cf-b6b3-d5bc-8196-b43cb31df495) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.5 [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [29]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    wp_1 = wp.join(
        lookup.withColumnRenamed("LocationID", "PULocationID"),
        "PULocationID", 
        "inner") \
        .select(col("Zone"), col("Borough"), 
                col("weighted_profit").cast("decimal(24,10)")) \
        .orderBy(desc("weighted_profit")) \
        .limit(20)
    
    return wp_1

Calculation started (calculation_id=aec955cf-b821-8a0b-8f41-cca72f3ef49b) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [30]:
main("large", "s3://cse6242-hpavlovich3/test2")

Calculation started (calculation_id=20c955cf-b95c-6dae-6846-a832058207c5) in (session=a6c955c6-3c3b-47d9-74b1-fcb50e9e24bf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
hpavlovich3
Trip Count:  187203269
+--------------------+-------------+----------------+
|                Zone|      Borough| weighted_profit|
+--------------------+-------------+----------------+
|Governor's Island...|    Manhattan|   -3.3375634432|
|    Great Kills Park|Staten Island|   -9.8752114804|
|       Rikers Island|        Bronx|  -43.3809220539|
|   Rossville/Woodrow|Staten Island| -114.0027492828|
|       Broad Channel|       Queens| -213.1493226310|
|     Freshkills Park|Staten Island| -244.5008453927|
|             Oakwood|Staten Island| -298.7605745251|
|Eltingville/Annad...|Staten Island| -315.5848130603|
|         Great Kills|Staten Island| -338.4961920198|
|Governor's Island...|    Manhattan| -468.0129006534|
| Green-Wood Cemetery|     Brooklyn| -495.0359547292|
|New Dorp/Midland ...|Staten Island| -535.2390003107|
|       West Brighton|Staten Island| -575.6133656888|
|Breezy Point/Fort...|       Queens| -679.8995329323|
|         Jamaica Bay|  